In [ ]:
'''
Get all audio, annos, beats, dataset path.
'''
import jams
removed = [13, 27, 40, 54, 82, 96, 109, 110, 120]
removed.extend(list(range(121, 151))) # album 10
removed.extend([167])
wave_removed = [13, 27, 40, 54, 82, 96, 109, 110, 120, 137]

audios_old = jams.util.find_with_extension('/userhome/cs2/u3556571/datasets/beatles/audio', 'mp3')
audios = jams.util.find_with_extension('/userhome/cs2/u3556571/latest_audio', 'wav')
audios = [audio for i, audio in enumerate(audios) if i not in wave_removed]
annos_old = jams.util.find_with_extension('/userhome/cs2/u3556571/datasets/beatles/annotations/chordlab', 'lab')
annos = jams.util.find_with_extension('/userhome/cs2/u3556571/datasets/beatles/annotations/chordlab2', 'lab')
annos = [anno for i, anno in enumerate(annos) if i not in wave_removed]
beats = jams.util.find_with_extension('/userhome/cs2/u3556571/datasets/beatles/annotations/beat', 'txt')
npzs_2nd = jams.util.find_with_extension('/userhome/cs2/u3556571/secondtry', 'npz')
npzs_3rd = jams.util.find_with_extension('/userhome/cs2/u3556571/3rd_wave/npzs', 'npz')
npzs_4th = jams.util.find_with_extension('/userhome/cs2/u3556571/4th_common_chord/npzs', 'npz')
npzs_5th = jams.util.find_with_extension('/userhome/cs2/u3556571/5th_majmin/npzs', 'npz')
npzs_6th = jams.util.find_with_extension('/userhome/cs2/u3556571/6th_fine_tuned_common/npzs', 'npz')
npzs_7th = jams.util.find_with_extension('/userhome/cs2/u3556571/7th/npzs', 'npz')
npzs_8th = jams.util.find_with_extension('/userhome/cs2/u3556571/8th/npzs', 'npz')

In [ ]:
'''
Constant variable declaration.
'''
PITCH_CLASS = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
FULL_CHORD_CLASS = ['maj', 'min', 'dim', 'aug', 'maj7', 'min7', '7', 'dim7', 'hdim7',
                    'minmaj7', 'maj6', 'min6', '9', 'maj9', 'min9', 'sus2', 'sus4',]
COMMON_CHORD_CLASS = ['maj', 'min']
MAJ_LIST = ['maj', 'maj7', '7', 'maj6', '9', 'maj9']
MIN_LIST = ['min', 'min7', 'minmaj7', 'min6', 'min9']
NO_LIST = ['dim', 'aug', 'dim7', 'hdim7', 'sus2', 'sus4']

In [ ]:
'''
Load audio.
'''
import librosa
def loadAudio(path):
    x, sr = librosa.load(path)
    return x, sr

In [ ]:
'''
Load .lab file.
Return with list of [timestamps with chord].
'''
def loadLab(labPath):
    chords = []
    with open(labPath, 'r') as f:
        for line in f.readlines():
            slot = line.rstrip().split()
            chords.append([float(slot[0]), float(slot[1]), slot[2]])
    return chords

In [ ]:
'''
Display audio.
'''
from IPython.display import Audio
from IPython.core.display import display
def displayAudio(x, sr):
    display(Audio(x, rate=sr))

In [ ]:
'''
Plot graph.
'''
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plotGraph(img):
    plt.figure()
    plt.imshow(img, cmap=plt.cm.gray_r)

def plotImgList(imgList, idx, shape):
    for img in imgList[(idx-1)*20:idx*20]:
        plotGraph(img.reshape(shape))

In [ ]:
'''
Convert 205 full chord labels to index.

Input:
    Chord label

Return:
    Corresponding chord index
'''
def fullChordToIdx(tag):
    if(tag=='N'):
        return 204
    
    tag = _standardizeChord(tag)
    items = tag.split(':')
    pitchIdx = PITCH_CLASS.index(items[0])
    chordIdx = FULL_CHORD_CLASS.index(items[1])
    
    return pitchIdx*17 + chordIdx

In [ ]:
'''
Convert 205 full chord index to labels.

Input:
    Chord label index

Return:
    Corresponding chord label
'''
def fullIdxToChord(idx):
    if (idx == 204):
        return 'N'
    return PITCH_CLASS[idx//17] + ':' + FULL_CHORD_CLASS[idx%17]

In [ ]:
'''
Convert 25 common chord labels to index.

Input:
    Chord labels

Return:
    Corresponding chord index
'''
def commonChordToIdx(tag):
    if(tag=='N'):
        return 24
    
    tag = _standardizeChord(tag)
    items = tag.split(':')
    pitchIdx = PITCH_CLASS.index(items[0])
    chordIdx = 0
    if (items[1] in MAJ_LIST):
        chordIdx = 0
    elif (items[1] in MIN_LIST):
        chordIdx = 1
    elif (items[1] in NO_LIST):
        return 24
    
    return pitchIdx*2 + chordIdx

In [ ]:
'''
Convert 25 common chord index to labels.

Input:
    Chord index

Return:
    Corresponding chord label
'''
def commonIdxToChord(idx):
    if (idx == 24):
        return 'N'
    return PITCH_CLASS[idx//2] + ':' + COMMON_CHORD_CLASS[idx%2]

In [ ]:
'''
Convert 3 maj, min chord labels to index.

Input:
    Chord labels

Return
    Corresponding chord index
'''
def majminChordToIdx(tag):
    if(tag=='N'):
        return 2
    
    tag = _standardizeChord(tag)
    items = tag.split(':')
    if (items[1] in MAJ_LIST):
        chordIdx = 0
    elif (items[1] in MIN_LIST):
        chordIdx = 1
    elif (chordIdx in NO_LIST):
        return 2
    
    return chordIdx

In [ ]:
'''
Convert 3 maj, min chord index to labels.

Input:
    Chord index

Return:
    Corresponding chord label
'''
def majminIdxToChord(idx):
    if (idx == 2):
        return 'N'
    return COMMON_CHORD_CLASS[idx%2]

In [ ]:
'''
Standardize chord label by dropping chord inversions and suppressed or additional notes.

Input:
    Chord label
    
Return:
    N or simplified chord label
'''
def _standardizeChord(tag):
    if (tag == 'N'):
        return 'N'
    
    splitted = tag.split(':')
    
    pitch = splitted[0].split('/')[0]
    if (pitch == 'Ab'):
        pitch = 'G#'
    elif (pitch == 'Bb'):
        pitch = 'A#'
    elif (pitch == 'Db'):
        pitch = 'C#'
    elif (pitch == 'Eb'):
        pitch = 'D#'
    elif (pitch == 'Gb'):
        pitch = 'F#'
    
    chord = None
    if(len(splitted)==1):
        chord = 'maj'
    else:
        chord = splitted[1].split('(')[0].split('/')[0]
        if (chord == ''):
            chord = 'maj'
    
    return pitch + ':' + chord